<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/edo/blob/main/clases/class_15/7_6_sist_laplace.ipynb#scrollTo=fb4c467b" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marco-canas/edo/blob/main/clases/class_15/7_6_sist_laplace.ipynb#scrollTo=fb4c467b"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# Un PVI de orden 2 solucionado con transformada de Laplace

$$ y'' -4y' = 6e^{3t} -3e^{-t}, \ \ \ y(0) = 1, \ \ \ y'(0) = -1 $$

## [Video de apoyo]() 

## Solución con sympy:

In [ ]:
import numpy as np 
from sympy import *

t = symbols('t', real = True, nonegative = True) 
s = symbols('s', real = True, positive = True) 

x,X,y,Y= symbols('x,X,y,Y')

x = Function('x')
y = Function('y')

X = Function('X')
Y = Function('Y') 


def L(f):
    return laplace_transform(f,t,s, noconds = True)  

def inv_L(F):
    return inverse_laplace_transform(F,s,t, noconds = True)  

def L_diff(f, f0, f_p0, f_pp0 = 0 , n = 1): # esta función es para calcular transformadas de derivadas
    """
    input:
    f es una variable sinbólica de sympy que representa a la función f(t) a la que se quiere calcular 
      la transformada de alguna de sus derivadas. 
    
    n puede asumir los valores de 0,1,2,3 y representa el orden de la derivada
    """
    F = str(f).upper()
    F = symbols(F) 
    F = Function(F)(s) 
    if n == 0:
        L = F
    elif n == 1:
        L = s*F - f0
    elif n == 2:
        L = s**2*F - s*f0 - f_p0
    elif n == 3:
        L = s**3*F - s**2*f0,-s*f_p0 - f_pp0
    return L 

Empecemos calculando la transformada de la ecuación diferencial dada:

In [ ]:
eq = Eq( diff(y(t),t,2) - 4*diff(y(t),t), 6*exp(3*t) - 3*exp(-t) )
eq 

In [ ]:
y0  = 1
yp0 = -1 

In [ ]:
eq_laplace = Eq( L_diff(y,y0,yp0,n = 2) - 4*L_diff(y, y0, yp0, n = 1) , 6*L(exp(3*t)) - 3*L(exp(-t)) ) 

In [ ]:
eq_laplace 

In [ ]:
sol = solve(eq_laplace,Y(s))[0]

In [ ]:
sol 

In [ ]:
def factor_2(s):
    return s**3 - 6*s**2 + 5*s + 12 

In [ ]:
import numpy as np 
posibles_raices_racionales = np.array([1,2,3,4,6,12, -1,-2,-3,-4,-6,-12])

In [ ]:
factor_2(posibles_raices_racionales) 

In [ ]:
Y = (s**3 - 7*s**2 + 10*s + 30)/(s*(s-3)*(s-4)*(s+1))
Y 

In [ ]:
Y = Y.apart() 
Y 

In [ ]:
y = inv_L(Y)
y 

In [ ]:
y_ = (11*exp(5*t) - 20*exp(4*t) + 25*exp(t) - 6)*exp(-t)/10 
y_ 

In [ ]:
eq.lhs.subs({y: y_ }) 

In [ ]:
eq.rhs

In [ ]:
y.subs({t:0}) 

In [ ]:
diff(y,t).subs({t:0}) 

## Referentes  

* Zill
